In [64]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, Flatten, Dense, Concatenate, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras import regularizers

In [65]:
NUM_INJURY_CATEGORIES = 28

injury_counts_input = Input(shape=(NUM_INJURY_CATEGORIES,), name='injury_counts')
position_input = Input(shape=(1,), name='position')
numerical_input = Input(shape=(6,), name='numerical') # height, weight, age

In [66]:
x_position = Embedding(input_dim=29, output_dim=4, input_length=1)(position_input) # train encoding on position index
x_position = Flatten()(x_position)

x_injury = Dense(128, activation='relu')(injury_counts_input)
x_injury = Dropout(0.15)(x_injury)
x_num = Dense(128, activation='relu')(numerical_input)
x_num = Dropout(0.15)(x_num)

x = Concatenate()([x_position, x_injury, x_num])

x = Dense(128, activation='relu')(x) # hidden layers for prediction
x = Dropout(0.3)(x)
x = Dense(64, activation='relu')(x)
x = Dropout(0.3)(x)

output = Dense(NUM_INJURY_CATEGORIES, activation='softmax')(x)

model = Model(inputs=[injury_counts_input, position_input, numerical_input], outputs=output)
model.summary()

/opt/anaconda3/envs/hacklytics/lib/python3.11/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "functional_9"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ position            │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ injury_counts       │ (None, 28)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ numerical           │ (None, 6)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_9         │ (None, 1, 4)      │        116 │ position[0][0]    │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_45 (Dense)    │ (None, 128)       │      3,712 │ injury_counts[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_46 (Dense)    │ (None, 128)       │        896 │ numerical[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_9 (Flatten) │ (None, 4)         │          0 │ embedding_9[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_27          │ (None, 128)       │          0 │ dense_45[0][0]    │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_28          │ (None, 128)       │          0 │ dense_46[0][0]    │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_9       │ (None, 260)       │          0 │ flatten_9[0][0],  │
│ (Concatenate)       │                   │            │ dropout_27[0][0], │
│                     │                   │            │ dropout_28[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_47 (Dense)    │ (None, 128)       │     33,408 │ concatenate_9[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_29          │ (None, 128)       │          0 │ dense_47[0][0]    │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_48 (Dense)    │ (None, 64)        │      8,256 │ dropout_29[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_30          │ (None, 64)        │          0 │ dense_48[0][0]    │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_49 (Dense)    │ (None, 28)        │      1,820 │ dropout_30[0][0]  │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 48,208 (188.31 KB)

 Trainable params: 48,208 (188.31 KB)

 Non-trainable params: 0 (0.00 B)

In [67]:
## Training
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.utils import to_categorical

df = pd.read_csv('data.csv')
injury_counts_cols = [col for col in df.columns if col.startswith('prev_')]
numerical_cols = ['height', 'weight', 'age', 'forty', 'bench', 'vertical']

scaler = StandardScaler()
df[numerical_cols] = scaler.fit_transform(df[numerical_cols]) # Normalize numerical data

injury_counts_data = df[injury_counts_cols].values
position_data = df['position_index']
numerical_data = df[numerical_cols].values

y = to_categorical(df["injury_index"], num_classes=NUM_INJURY_CATEGORIES)

(X_injury_counts_train, X_injury_counts_val_test, X_position_train, X_position_val_test, X_numerical_train, X_numerical_val_test, y_train, y_val_test) = train_test_split(injury_counts_data, position_data, numerical_data, y, test_size=0.3)

(X_injury_counts_val, X_injury_counts_test, X_position_val, X_position_test, X_numerical_val, X_numerical_test, y_val, y_test) = train_test_split(X_injury_counts_val_test, X_position_val_test, X_numerical_val_test, y_val_test, test_size=0.1)

model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

In [68]:
from tensorflow.keras.callbacks import ReduceLROnPlateau

model.fit(
    x=[X_injury_counts_train, X_position_train, X_numerical_train],
    y=y_train,
    validation_data=([X_injury_counts_val, X_position_val, X_numerical_val], y_val),
    epochs=1000,
    batch_size=128,
)

model.save('model.h5')

Epoch 1/1000


48/48 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.1388 - loss: 3.1172 - val_accuracy: 0.2924 - val_loss: 2.5288
Epoch 2/1000
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.2570 - loss: 2.6187 - val_accuracy: 0.3289 - val_loss: 2.4084
Epoch 3/1000
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.2862 - loss: 2.4982 - val_accuracy: 0.3508 - val_loss: 2.3242
Epoch 4/1000
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.3189 - loss: 2.4164 - val_accuracy: 0.3564 - val_loss: 2.2727
Epoch 5/1000
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.3282 - loss: 2.3671 - val_accuracy: 0.3667 - val_loss: 2.2309
Epoch 6/1000
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.3417 - loss: 2.3142 - val_accuracy: 0.3607 - val_loss: 2.1983
Epoch 7/1000
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3350 - loss: 2.3012 - val_accuracy: 0.3697 - val_loss: 2.1705
Epoch 8/1000
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3526 - loss: 2.2548 - val_accuracy: 0.3757 - val_lo

In [69]:
# Test

test_loss, test_acc = model.evaluate(
    x=[X_injury_counts_test, X_position_test, X_numerical_test],
    y=y_test,
    verbose=1
)

print('Test Loss:', test_loss)
print('Test Accuracy:', test_acc)

9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4406 - loss: 2.5271 
Test Loss: 2.5066893100738525
Test Accuracy: 0.44015443325042725
